# Amazonia-1 on AWS Open Data

## Accessing Cloud Optimized Multi-Band Raster Data


With the AWS ASDI STAC API, we can search and load various geospatial datasets. Below we'll work with the Amazonia-1 Satelite WFI Camera, Level-4 Orthorectified data sourced from Cloud Optimized GeoTIFFs (COGs) [hosted on AWS](https://registry.opendata.aws/amazonia/). In this example, we'll use `TiTiler` create interactive visualizations of the Amazonia-1 dataset.

### Dependencies

First we install any necessary packages. Please note that the following block of code only needs to be run if needed within a new workspace and that you may need to restart the kernel to use updated packages. The code block only needs to be run once for each AWS Sagemaker Studio profile.

In [2]:
%pip install --upgrade pip -q
%pip install matplotlib 'pystac_client>=0.5.0' stackstac rasterio folium httpx shapely -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:
# All imports in the order they are used in the code
from pystac_client import Client
import folium
from shapely.geometry import box
from shapely.ops import unary_union
import httpx
from IPython.display import JSON

We'll use `pystac_client` to submit our search query, stackstac to load and composite our rasters as an Xarray DataArray, and rio-tiler to performantally load individual COGs.

### Querying the AWS STAC API with Pystac Client

First, we define the URL to our STAC API, a service that allows us to submit search queries for spatio-temporal assets (like elevation raster tiles).

Next, we open this URL with `pystac_client`, which allows us to use the client's search method. `client.search` accepts many different arguments to specify the search query. Common query parameters are `time_range` and `bbox` (the spatial bounding box). Since the DEM was collected at one point in time, we will only specify the "where" with a `bbox` describing the minx, miny, maxx, maxy coordinates in lat/lon degrees.

In [18]:
# making a connection to AWS STAC API
API_ROOT_URL = "https://dev.asdi-catalog.org"
client = Client.open(API_ROOT_URL)

# Search for the AMAZONIA1-WF1 tiles from a specific date range
source_search = client.search(
    collections=["AMAZONIA1-WFI"], 
    datetime=['2023-06-25T00:00:00Z', '2023-06-29T00:00:00Z'],
    max_items=100
)
# Getting the metadata and links to data assets in STAC format
source_items = source_search.item_collection()
print("Number of WFI tiles: ", len(source_items))
source_items

Number of WFI tiles:  39


### Overview Map for Context

To better understand our search results, we can create a descriptive map to display their bounding boxes. We'll load an OpenStreetMap basemap that includes place names, annotations for water and forested areas, and roads. We'll use `folium` to access and plot this map. To learn more about `folium`, check out the [documentation](https://python-visualization.github.io/folium/). We'll use our bounding box to specify where we want to center our map and pick a zoom level that shows us detail around San Ignacio.

In [43]:
# Use shapely to find the bounds of the query results
result_bounds = unary_union([box(*item.bbox) for item in source_items])
[west, south, east, north] = result_bounds.bounds

# Use Folium to create an interactive map to show the query and results boxes
m = folium.Map()
m.fit_bounds([[south, west], [north, east]])
bbox_style_blue = {'fillColor': '#3388ff', 'color': '#3388ff', 'fillOpacity':'0.05'} #make bbox blue
folium.GeoJson(source_items.to_dict(), name="items", style_function=lambda x:bbox_style_blue).add_to(m)
m

### Add the search results to the map

We'll use [`TiTiler.PgSTAC`](https://stac-utils.github.io/titiler-pgstac/intro/) to save our query as a "mosaic" of the 20 overlapping tiles our search found earlier. `TiTiler.PgSTAC` will save our search in a database so that we can use an identifier to refer to the results. So first we create a dictionary describing the search and some visualization options we'll use with the map layer.


In [39]:
# Take the search parameters, add filter language, and some visualization defaults
search_request = { 
    **source_search.get_parameters(),
    "filter-lang": "cql-json",
    "metadata": {
        "minzoom": 8,
        "maxzoom": 13,
        # Set some color schemes to be able to view them
        "defaults": {
            "true_color" : {
                "assets": ["B3", "B2", "B1"],
                "color_formula": "Gamma RGB 3.5 Saturation 1.7 Sigmoidal RGB 15 0.35",
            },
            "false_color_ir" : {
                "assets": ["B4", "B3", "B2"],
            },
            "ndvi": {
                "expression": "(B4-B3)/(B4+B3)",
                "rescale": "-1,1",
                "colormap_name": "viridis"
            }
        }
    }
}
JSON(search_request)

<IPython.core.display.JSON object>

In [8]:
# Set the API endpoint for the Tiler and check that it's up
endpoint = "https://titiler.dev.asdi-catalog.org"
JSON(httpx.get(f"{endpoint}/healthz").json())

<IPython.core.display.JSON object>

In [41]:
# Register the STAC query as a mosaic
response = httpx.post(
    f"{endpoint}/mosaic/register", json=search_request,
).json()
searchid = response["searchid"]
JSON(response)

<IPython.core.display.JSON object>

Now that the search is persisted in [`TiTiler.PgSTAC`](https://stac-utils.github.io/titiler-pgstac/intro/), we can start working on adding the tileset to our map. We need to build a tilejson url to the tile server that includes the rendering options we'd like `TiTiler` to use to create our tileset.

We can ask `TiTiler` what assets are available from our serach results. To do this, we use a point in the bounding box of one of the resulting items (in this case the centroid) to query the `TiTiler` api for assets.

In [48]:
center = box(*source_items[0].bbox).centroid
tj_response = httpx.get(f"{endpoint}/mosaic/{searchid}/{center.x},{center.y}/assets").json()
print(tj_response[0]["assets"].keys())

dict_keys(['B1', 'B2', 'B3', 'B4', 'metadata', 'thumbnail'])


These `B#` assets are bands of our dataset. To get a useable tileset, we ask `TiTiler` to combine these bands into images. Here we'll ask `TiTiler` to create a tileset that uses the first three bands to create a color image.

In [50]:

params = {
    "assets": ["B3", "B2", "B1"],
    "color_formula": "Gamma RGB 3.0 Saturation 1.7 Sigmoidal RGB 20 0.19",
}
tj_response = httpx.get(
  f"{endpoint}/mosaic/{searchid}/tilejson.json",
  params=params
).json()
JSON(tj_response)

<IPython.core.display.JSON object>

In [54]:
# Make another folium map
m = folium.Map()
m.fit_bounds([[south, west], [north, east]])

# Add the tile later
aod_layer = folium.TileLayer(
    tiles=tj_response["tiles"][0],
    attr="Mosaic",
    min_zoom=4,
    max_zoom=18,
    max_native_zoom=18,
)
aod_layer.add_to(m)
m

`TiTiler` can combine the bands in much more intersting ways like visualizing [NVDI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) by applying a formula to the bands. 

In [56]:
# Request a tilejson using an expression of bands
params = {
  'expression': '(B4-B3)/(B4+B3)',
  'asset_as_band': True,
  'rescale': '-1,1',
  'colormap_name': 'viridis'
}
tj_response = httpx.get(
  f"{endpoint}/mosaic/{searchid}/tilejson.json",
  params=params
).json()

# Make another folium map
m = folium.Map()
m.fit_bounds([[south, west], [north, east]])

# Add the tile later
aod_layer = folium.TileLayer(
    tiles=tj_response["tiles"][0],
    attr="Mosaic",
    min_zoom=4,
    max_zoom=18,
    max_native_zoom=18,
)
aod_layer.add_to(m)
m